In [1]:
!pip install openai-whisper transformers speechrecognition pydub torch pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/

In [2]:
!pip install -q openai-whisper
import whisper


In [3]:
!pip install gtts


In [5]:
# Importing necessary libraries
!pip install deep-translator

import whisper
import speech_recognition as sr
from transformers import pipeline
from gtts import gTTS
import IPython.display as ipd
import random
from deep_translator import GoogleTranslator
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import nltk
import librosa
import numpy as np

nltk.download('punkt')

# Loading AI Models
print("\n Loading AI models... Please wait.")
asr_model = whisper.load_model("base")
sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
print("✅ Models Loaded!")

# History Tracking
history_data = pd.DataFrame(columns=["Date", "Mood", "Recommendations"])
user_mood_scores = {}

# Let the User Choose Their Language
language_map = {
    "1": "en", "2": "es", "3": "fr", "4": "hi", "5": "te", "6": "kn",
    "7": "ta", "8": "bn", "9": "gu", "10": "mr", "11": "ml", "12": "pa",
    "13": "ur", "14": "ar", "15": "ru", "16": "zh", "17": "ja", "18": "ko",
    "19": "de", "20": "pt"
}

print("\n🌎 Choose a language for AI responses:")
for key, value in language_map.items():
    print(f"{key}️⃣ {value.upper()}")

language_choice = input("Enter your choice (1-20): ")
ai_language = language_map.get(language_choice, "en")

# Function to Translate Recommendations
def translate_recommendations(recommendations, target_lang):
    return [GoogleTranslator(source="en", target=target_lang).translate(text) for text in recommendations]

# Function to Speak AI Responses in the User's Chosen Language
def speak(mood, recommendations, lang="en"):
    print(f"🗣️ AI Speaking in: {lang}")
    if lang != "en":
        recommendations = translate_recommendations(recommendations, lang)
    text = GoogleTranslator(source="en", target=lang).translate(
        f"I detected that you are feeling {mood}. Here are my recommendations: {', '.join(recommendations)}."
    )

    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        tts.save("response.mp3")
        return ipd.Audio("response.mp3", autoplay=True)
    except:
        print("❌ Language not supported! Switching to English.")
        tts = gTTS(text=text, lang="en")
        tts.save("response.mp3")
        return ipd.Audio("response.mp3", autoplay=True)

# Advanced Sentiment Analysis
def analyze_sentiment(text):
    sentiment = sentiment_analysis(text)[0]['label']
    if any(word in text.lower() for word in ["stress", "pressure", "tension"]):
        return "stressed"
    if "5 stars" in sentiment:
        return "happy"
    elif "4 stars" in sentiment:
        return "neutral"
    elif "3 stars" in sentiment:
        return "anxious"
    elif "2 stars" in sentiment:
        return "sad"
    else:
        return "stressed"

# Mood Prediction
def predict_mood():
    return random.choice(["happy", "sad", "anxious", "stressed", "neutral"])

# Function to Recommend Unique Activities Based on Mood
def recommend_activity(user_mood):
    recommendations_map = {
        'happy': ['Music Therapy', 'Outdoor Activities', 'Dancing', 'Journaling', 'Cooking a New Recipe'],
        'sad': ['Cognitive Behavioral Therapy (CBT)', 'Journaling', 'Guided Meditation', 'Listening to Calming Music'],
        'anxious': ['Breathing Exercises', 'Yoga', 'Mindfulness Techniques', 'Stretching Routines', 'Gardening'],
        'stressed': ['Therapy Sessions', 'Workout', 'Relaxation Music', 'Tai Chi', 'Deep Breathing'],
        'neutral': ['Nature Walks', 'Reading Books', 'Aromatherapy', 'Watching a Stand-up Comedy Show']
    }
    return random.sample(recommendations_map.get(user_mood.lower(), ["Meditation"]), 3)

# Voice Tone Analysis
def analyze_voice_tone(audio_file):
    y, sr = librosa.load(audio_file)
    pitch = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    if pitch > 2500:
        return "stressed"
    elif pitch > 2000:
        return "anxious"
    else:
        return "neutral"

# Function to Process Voice Input
import speech_recognition as sr

#  Function to Capture Live Speech in Google Colab
from google.colab import files

# Function to Upload & Process Voice Input in Google Colab
def record_voice():
    print("\n🎤 Upload an audio file (WAV or MP3)...")

    # Let user upload an audio file
    uploaded = files.upload()

    # Get the first uploaded file
    audio_file = list(uploaded.keys())[0]

    print(f"✅ Audio file '{audio_file}' uploaded successfully! Processing...")

    try:
        # Use Whisper AI to transcribe audio
        result = asr_model.transcribe(audio_file)
        transcribed_text = result["text"]

        print(f"📝 Transcribed Text: {transcribed_text}")

        # Analyze sentiment from the transcribed text
        sentiment = analyze_sentiment(transcribed_text)
        recommendations = recommend_activity(sentiment)

        # Speak the recommendations in the selected language
        display(speak(sentiment, recommendations, ai_language))

        return transcribed_text, sentiment, recommendations

    except Exception as e:
        print(f"❌ Error processing audio: {str(e)}")



#Mood Trends & History Visualization
def analyze_mood_trends():
    plt.figure(figsize=(10,5))
    plt.plot(list(user_mood_scores.keys()), list(user_mood_scores.values()), marker='o', linestyle='-')
    plt.xlabel("Date")
    plt.ylabel("Mood Score (1-10)")
    plt.title("Mood Trends Over Time")
    plt.show()

# User Input Menu
print("\n🔹 Choose an option:")
print("1️⃣ Enter a text-based mood description")
print("2️⃣ Speak into the microphone")
print("3️⃣ View Mood Trends & History")
print("4️⃣ Predict My Next Mood")
print("5️⃣ AI Guided Meditation")

choice = input("Enter 1-5: ")

if choice == "1":
    user_text = input("📝 Enter your mood: ")
    sentiment = analyze_sentiment(user_text)
    recommendations = recommend_activity(sentiment)
    display(speak(sentiment, recommendations, ai_language))

elif choice == "2":
    record_voice()

elif choice == "3":
    analyze_mood_trends()

elif choice == "4":
    predicted_mood = predict_mood()
    print(f"\n🔮 AI Predicts Your Next Mood: {predicted_mood}")

elif choice == "5":
    display(speak("neutral", ["Take deep breaths and relax."], ai_language))

else:
    print("❌ Invalid choice! Please enter 1-5.")

print("\n🚀 AI Mental Health Assistant 2025! ")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



 Loading AI models... Please wait.


Device set to use cpu


Models Loaded!

 Choose a language for AI responses:
1️⃣ EN
2️⃣ ES
3️⃣ FR
4️⃣ HI
5️⃣ TE
6️⃣ KN
7️⃣ TA
8️⃣ BN
9️⃣ GU
10️⃣ MR
11️⃣ ML
12️⃣ PA
13️⃣ UR
14️⃣ AR
15️⃣ RU
16️⃣ ZH
17️⃣ JA
18️⃣ KO
19️⃣ DE
20️⃣ PT
Enter your choice (1-20): 4

🔹 Choose an option:
 Enter a text-based mood description
 Speak into the microphone
 View Mood Trends & History
 Predict My Next Mood
 AI Guided Meditation
Enter 1-5: 2

 Upload an audio file (WAV or MP3)...


Saving recorded_audio.wav to recorded_audio (1).wav
 Audio file 'recorded_audio (1).wav' uploaded successfully! Processing...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Transcribed Text:  I'm very happy now.
 AI Speaking in: hi



 AI Mental Health Assistant 2025! 
